Konsta Nyman <br>
523834 <br>
January, 29, 2024  <br>

# Exercise 2 | TKO_7092 Evaluation of Machine Learning Methods 2024

### Prediction of the metal ion content from multi-parameter data
<b> Use K-Nearest Neighbor Regression with euclidean distance to predict total metal concentration (c_total), concentration of Cadmium (Cd) and concentration of Lead (Pb), using number of neighbors k = 1, 3, 5, 7.</b> <br>

<b> Instructions: </b> 

    - You may use Nearest Neighbor Regression from https://scikit-learn.org/stable/modules/neighbors.html
    - The data should be standarized using z-score (using sklearn.preprocessing.StandardScaler is advised).
    - Implement Leave-One-Out cross-validation and calculate the C-index for each output (c_total, Cd, Pb). 
    - Implement Leave-Replicas-Out cross-validation and calculate the C-index for each output (c_total, Cd, Pb).
    - Explain your code by adding detailed comments. 
    - Only provide code that is relevant to the exercise.
    - Please submit your solution as a Jupyter Notebook (.ipynb) and as a PDF file. Ensure to include your full name in the filename.     
    - Submit to moodle your solution on ** Wednesday 7 of February ** at the latest.
  
<b>Please follow the instructions and note that you are expected to submit your individual solution.<br>
Identical or overly similar submissions will result in the exercise being marked as failed.
</b>

## Import libraries

In [24]:
# In this cell import all libraries you need. For example: 
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeaveOneGroupOut


## Read and visualize the dataset

In [25]:
# In this cell read the file Water_data.csv
df_water_data = pd.read_csv('water_data.csv')

# Print the dataset dimesions (i.e. number of rows and columns)
print(f"The dimensions of the dataset are {df_water_data.shape}\n")

# Print the first 5 rows of the dataset
print(f"The first 5 rows of the dataset:\n\n{df_water_data.head()}")

The dimensions of the dataset are (268, 6)

The first 5 rows of the dataset:

    Mod1  Mod2    Mod3  c_total   Cd    Pb
0   9945   119   72335        0  0.0   0.0
1   9596   119  110542        0  0.0   0.0
2  10812   120   98594        0  0.0   0.0
3  10786   117   82977        0  0.0   0.0
4  10566   108  136416       14  0.0  14.0


## Standardization of the dataset

In [34]:
# In this cell, standardize the dataset features by removing the mean and scaling to unit variance. 
# In other words, use z-score to scale the dataset features (Mod1, Mod2, Mod3)

# create varaibles of input and output data column names
input_columns = ['Mod1', 'Mod2', 'Mod3']
output_columns = ['c_total', 'Cd', 'Pb']

scaler = StandardScaler() #create a scaler
df_water_data[input_columns] = scaler.fit_transform(df_water_data[input_columns]) #fit and transform the selected columns

# Print the 5 first samples (i.e. rows) of the scaled dataset
df_water_data.head()

,Mod1,Mod2,Mod3,c_total,Cd,Pb
0,-0.972283,-0.670482,-0.358179,0,0.0,0.0
1,-0.975878,-0.670482,0.259488,0,0.0,0.0
2,-0.963351,-0.670394,0.066333,0,0.0,0.0
3,-0.963619,-0.670657,-0.186137,0,0.0,0.0
4,-0.965885,-0.671447,0.677776,14,0.0,14.0


## C-index code 

In [ ]:
# In this cell, implement the C-index function. You may use the implementation from the first exercise.
def cindex(true_labels, pred_labels):
    """Returns C-index between true labels and predicted labels"""         
    return cindx

In [ ]:
# Test the cindex function with following values
true_labels = np.array([-1, 1, 1, -1, 1])
predictions = np.array([0.60, 0.80, 0.75, 0.75, 0.70])
cindx = cindex(true_labels, predictions)
print(cindx) #For this example, a correct C-index implementation will result in 0.75

NameError: name 'cindx' is not defined

## Leave-One-Out cross-validation
In the following cell, write and execute your code for Leave-One-Out cross-validation using K-Nearest Neighbor Regression with k values of 1, 3, 5, and 7.<br>
Print the corresponding Leave-One-Out C-index for c_total, Cd and Pb for each k value.

## Leave-Replicas-Out cross-validation

In the following cell, write and execute your code for Leave-Replicas-Out cross-validation using K-Nearest Neighbor Regression with k values of 1, 3, 5, and 7.<br>
Print the corresponding Leave-Replicas-Out C-index for c_total, Cd and Pb for each k value.

## Plot Leave-One-Out and Leave-Replicas-Out Results

Note: You may plot the results as they were presented in the video lecture (refer to MOOC2-Module 2 .pptx slides).

## Interpretation of results
#### Answer the following questions based on the results obtained
- Which cross-validation method had more optimistic results? 
- Explain the reason for the optimistic results produced by the cross-validation method.
- Which cross-validation method generalized better on unseen data? Why?

In [ ]:
#In this cell write your answers to the questions.